In [78]:
# Installing Googlemaps Package
!pip install googlemaps

In [79]:
# Importing Required Packages
import requests
import googlemaps
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
from IPython.display import display, HTML
from google.colab import files

In [80]:
# Function to Create a Scrollable Table Within a Small Window
def scroll_table(data, table_id, title):
    html = f'<h3>{title}</h3>'
    html += f'<div id="{table_id}" style="height:400px;overflow:auto;">'
    html += data.to_html()
    html += '</div>'
    return html

In [81]:
# Uploading the Google API Key Text File
upload = files.upload()

Saving Google_API_Key to Google_API_Key (1)


In [82]:
# Reading the File and Storing it as a Variable
with open('Google_API_Key', 'r') as file:
    google_API_Key = file.read().strip("'")

In [83]:
# Creating a GoogleMap Client
gmaps_client = googlemaps.Client(key=google_API_Key)

In [84]:
# Uploading the Clean Dataset
uploaded = files.upload()

Saving Plan_df.xlsx to Plan_df (1).xlsx


In [85]:
# Reading the Cleaned Dataset
truck_df = pd.read_excel('Plan_df.xlsx')

In [86]:
# Viewing the Resulting DataFrame
display_truck_df = truck_df
html_1 = scroll_table(display_truck_df, 'truck_df_html', "Clean Truck Data")
display(HTML(html_1))

,Food_Truck,Website,Phone_Number,Cuisine,Rating,Review_Count,Latitude,Longitude,Address,Start_Time,End_Time
0,The Alkaline Electric Goddess,https://www.yelp.com/biz/the-alkaline-electric-goddess-indianapolis?adjust_creative=7dFhg7WTQv5IwbOVO9QLzw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_lookup&utm_source=7dFhg7WTQv5IwbOVO9QLzw,(317) 662-4498,Health Markets,5.0,1,39.765204,-86.158881,"49 W Maryland St unit 118a, Indianapolis, IN 46204, USA",05:00 PM,09:00 PM
1,The Legendary Kitchen,https://www.yelp.com/biz/the-legendary-kitchen-indianapolis?adjust_creative=7dFhg7WTQv5IwbOVO9QLzw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_lookup&utm_source=7dFhg7WTQv5IwbOVO9QLzw,(800) 568-9370,Food Trucks,5.0,2,39.778465,-86.156783,"40 E St Clair St, Indianapolis, IN 46204, USA",09:00 AM,05:00 PM
2,La Familia,https://www.yelp.com/biz/la-familia-food-truck-indianapolis?adjust_creative=7dFhg7WTQv5IwbOVO9QLzw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_lookup&utm_source=7dFhg7WTQv5IwbOVO9QLzw,(463) 280-4499,Tacos,5.0,8,39.721966,-86.153505,"311 Troy Ct, Indianapolis, IN 46227, USA",11:00 AM,11:00 PM
3,Tapatia Mexican Tacos,https://www.yelp.com/biz/tapatia-mexican-tacos-warren-park?adjust_creative=7dFhg7WTQv5IwbOVO9QLzw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_lookup&utm_source=7dFhg7WTQv5IwbOVO9QLzw,(317) 490-2417,Tacos,5.0,1,39.772727,-86.040069,"7202 E Washington St, Indianapolis, IN 46219, USA",12:00 PM,11:00 PM
4,Traveling Taste Buds,https://www.yelp.com/biz/traveling-taste-buds-indianapolis?adjust_creative=7dFhg7WTQv5IwbOVO9QLzw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_lookup&utm_source=7dFhg7WTQv5IwbOVO9QLzw,(317) 531-6155,Food Trucks,5.0,1,39.768534,-86.158142,"1 Monument Cir, Indianapolis, IN 46204, USA",12:00 AM,12:00 AM
5,HomeTeam Ice Cream,https://www.yelp.com/biz/hometeam-ice-cream-kokomo?adjust_creative=7dFhg7WTQv5IwbOVO9QLzw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_lookup&utm_source=7dFhg7WTQv5IwbOVO9QLzw,(765) 437-6843,Food Trucks,5.0,2,40.492290,-86.130060,"709 N Union St, Kokomo, IN 46901, USA",04:00 PM,08:00 PM
6,Ice Cream On Wheels,https://www.yelp.com/biz/ice-cream-on-wheels-indianapolis-6?adjust_creative=7dFhg7WTQv5IwbOVO9QLzw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_lookup&utm_source=7dFhg7WTQv5IwbOVO9QLzw,(800) 884-9793,Food Trucks,5.0,1,39.691830,-86.234110,"4215 W Thompson Rd, Indianapolis, IN 46221, USA",12:00 AM,12:00 AM
7,Tacopolis,https://www.yelp.com/biz/tacopolis-indianapolis-2?adjust_creative=7dFhg7WTQv5IwbOVO9QLzw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_lookup&utm_source=7dFhg7WTQv5IwbOVO9QLzw,(317) 488-8724,Tacos,5.0,5,39.823769,-86.242204,"4857 W 38th St, Indianapolis, IN 46254, USA",02:00 PM,11:30 PM
8,TACO LOCO M&C,https://www.yelp.com/biz/taco-loco-m-and-c-indianapolis?adjust_creative=7dFhg7WTQv5IwbOVO9QLzw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_lookup&utm_source=7dFhg7WTQv5IwbOVO9QLzw,(463) 210-9667,Food Trucks,5.0,1,39.911651,-86.184593,"1415 W 86th St, Indianapolis, IN 46260, USA",06:00 PM,12:00 AM
9,Scout's Treat Truck,https://www.yelp.com/biz/scouts-treat-truck-indianapolis?adjust_creative=7dFhg7WTQv5IwbOVO9QLzw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_lookup&utm_source=7dFhg7WTQv5IwbOVO9QLzw,(317) 409-2600,Cupcakes,4.5,19,39.908417,-86.082016,"8375 Allison Pointe Blvd, Indianapolis, IN 46250, USA",06:30 AM,08:00 PM


In [87]:
# Extracting City from Addresses of Trucks

# Creating a Function to Extract the City from the Formatted Address
def extract_city(address):
    parts = address.split(',')
    if len(parts) > 2:
        return parts[-3].strip()
    else:
        return None

# Applying the Function on the DataFrame to Create a 'City' Column
truck_df['City'] = truck_df['Address'].apply(extract_city)

In [88]:
# Viewing Number of Food Trucks per City
print(truck_df['City'].value_counts())

Indianapolis    26
Kokomo           1
Shelbyville      1
Name: City, dtype: int64


In order to account for travel time and quality of food, I am filtering the data by the following:
* All trucks must be located in Indianapolis
* All trucks must have a minimum rating of 4 or higher

In [89]:
# Creating a DataFrame with Required Columns and Removing Cities Other Than Indianapolis
trucks_filtered_df = truck_df.drop(columns =
                                   ['Website', 'Phone_Number']).query("City == 'Indianapolis'").reset_index(drop=True).query('Rating >= 4.0')

In [90]:
# Viewing the Resulting DataFrame
display_trucks_filtered_df = trucks_filtered_df
html_2 = scroll_table(display_trucks_filtered_df, 'trucks_filtered_df_html', "Filtered Truck Data")
display(HTML(html_2))

,Food_Truck,Cuisine,Rating,Review_Count,Latitude,Longitude,Address,Start_Time,End_Time,City
0,The Alkaline Electric Goddess,Health Markets,5.0,1,39.765204,-86.158881,"49 W Maryland St unit 118a, Indianapolis, IN 46204, USA",05:00 PM,09:00 PM,Indianapolis
1,The Legendary Kitchen,Food Trucks,5.0,2,39.778465,-86.156783,"40 E St Clair St, Indianapolis, IN 46204, USA",09:00 AM,05:00 PM,Indianapolis
2,La Familia,Tacos,5.0,8,39.721966,-86.153505,"311 Troy Ct, Indianapolis, IN 46227, USA",11:00 AM,11:00 PM,Indianapolis
3,Tapatia Mexican Tacos,Tacos,5.0,1,39.772727,-86.040069,"7202 E Washington St, Indianapolis, IN 46219, USA",12:00 PM,11:00 PM,Indianapolis
4,Traveling Taste Buds,Food Trucks,5.0,1,39.768534,-86.158142,"1 Monument Cir, Indianapolis, IN 46204, USA",12:00 AM,12:00 AM,Indianapolis
5,Ice Cream On Wheels,Food Trucks,5.0,1,39.691830,-86.234110,"4215 W Thompson Rd, Indianapolis, IN 46221, USA",12:00 AM,12:00 AM,Indianapolis
6,Tacopolis,Tacos,5.0,5,39.823769,-86.242204,"4857 W 38th St, Indianapolis, IN 46254, USA",02:00 PM,11:30 PM,Indianapolis
7,TACO LOCO M&C,Food Trucks,5.0,1,39.911651,-86.184593,"1415 W 86th St, Indianapolis, IN 46260, USA",06:00 PM,12:00 AM,Indianapolis
8,Scout's Treat Truck,Cupcakes,4.5,19,39.908417,-86.082016,"8375 Allison Pointe Blvd, Indianapolis, IN 46250, USA",06:30 AM,08:00 PM,Indianapolis
9,Pico De Gallo,Food Trucks,4.5,6,39.853872,-86.231604,"4202 W 56th St, Indianapolis, IN 46254, USA",05:00 AM,01:00 PM,Indianapolis


In [91]:
# Viewing Spread of Ratings
trucks_filtered_df['Rating'].value_counts()

4.5    9
5.0    8
4.0    5
Name: Rating, dtype: int64

In [92]:
# Viewing Number of Cuisine Types
trucks_filtered_df['Cuisine'].value_counts()

Food Trucks       9
Tacos             4
Caterers          2
Health Markets    1
Cupcakes          1
Bakeries          1
Mexican           1
Comfort Food      1
Pizza             1
Sandwiches        1
Name: Cuisine, dtype: int64

As Yelp did not have specific Cuisine information about all the food trucks, I am manually coding some ambiguous Cuisines, to ensure the trip includes a variety of cuisines.

In [93]:
# Selecting Relevant Data to Ease Manual Coding Process
manual_df = trucks_filtered_df[['Food_Truck', 'Cuisine']].query('Cuisine == "Food Trucks" | Cuisine == "Caterers" | Cuisine == "Health Markets"').reset_index(drop=True)

In [94]:
# Viewing the Resulting DataFrame
display_manual_df = manual_df
html_3 = scroll_table(display_manual_df, 'manual_df_html', "Trucks with Ambiguous Cuisines")
display(HTML(html_3))

,Food_Truck,Cuisine
0,The Alkaline Electric Goddess,Health Markets
1,The Legendary Kitchen,Food Trucks
2,Traveling Taste Buds,Food Trucks
3,Ice Cream On Wheels,Food Trucks
4,TACO LOCO M&C,Food Trucks
5,Pico De Gallo,Food Trucks
6,El Pollo Tropical,Food Trucks
7,Fast Taco's,Food Trucks
8,Serendipity Mobile Catering,Caterers
9,Gaucho's Fire Express,Caterers


In [95]:
# Creating List of New Cuisines from Truck Websites
health_market_new_cuisine = 'Comfort Food'
food_truck_new_cuisine_list = ['Comfort Food', 'Comfort Food', 'Ice Cream', 'Tacos', 'Mexican', 'Mexican', 'Tacos', 'Comfort Food', 'Comfort Food']
catering_new_cuisine_list = ['Comfort Food', 'Sandwiches']

# Replacing 'Food Trucks' with New Cuisines
trucks_filtered_df['Cuisine'] = trucks_filtered_df['Cuisine'].replace({"Health Markets": health_market_new_cuisine,
                                                                       "Food Trucks": food_truck_new_cuisine_list,
                                                                       "Caterers": catering_new_cuisine_list})

# Viewing Value Counts for New Cuisines
trucks_filtered_df['Cuisine'].value_counts()

Comfort Food    7
Tacos           6
Mexican         3
Sandwiches      2
Ice Cream       1
Cupcakes        1
Bakeries        1
Pizza           1
Name: Cuisine, dtype: int64

A lot of the cuisines are very similar to each other, so I will group them into more general cuisines, by country or type of food

In [96]:
# Grouping Similar Cuisines
trucks_filtered_df['Cuisine'] = trucks_filtered_df['Cuisine'].replace({"Tacos": "Mexican",
                                                                       "Cupcakes": "Bakeries"})
# Viewing Value Counts for Grouped Cuisines
trucks_filtered_df['Cuisine'].value_counts()

Mexican         9
Comfort Food    7
Bakeries        2
Sandwiches      2
Ice Cream       1
Pizza           1
Name: Cuisine, dtype: int64

Ratings are a good indicator of quality, but if a truck has a rating of 5, but with only 1 review, it is likely not popular or does not deserve that rating. Thus, I will create a normalized rating column, by taking the following steps:
* Normalizing the 'Review Column' using Mean Normalization
* Multiply the 'Ratings' Column by the normalized reviews, to create the normalized ratings column

In [97]:
# Normalizing the Rating Column

# Defining a Function to Apply Mean Normalization to a column
def mean_normalization(column):
  mean = column.mean()
  min = column.min()
  max = column.max()

  # Mean Normalization Formula
  normalized_column = (column - mean) / (max - min)
  return normalized_column

# Creating a Normalized Rating Column
Normalized_Reviews = mean_normalization(trucks_filtered_df['Review_Count'])
trucks_filtered_df['Normalized_Ratings'] = trucks_filtered_df['Rating']*Normalized_Reviews

rated_trucks_df = trucks_filtered_df.drop(columns=['Rating', 'Review_Count', 'City'])

In [98]:
# Viewing the Resulting DataFrame
display_rated_trucks_df = rated_trucks_df
html_4 = scroll_table(display_rated_trucks_df, 'rated_trucks_df_html', "Truck Data with Normalized Ratings")
display(HTML(html_4))

,Food_Truck,Cuisine,Latitude,Longitude,Address,Start_Time,End_Time,Normalized_Ratings
0,The Alkaline Electric Goddess,Comfort Food,39.765204,-86.158881,"49 W Maryland St unit 118a, Indianapolis, IN 46204, USA",05:00 PM,09:00 PM,-1.067584
1,The Legendary Kitchen,Comfort Food,39.778465,-86.156783,"40 E St Clair St, Indianapolis, IN 46204, USA",09:00 AM,05:00 PM,-1.001794
2,La Familia,Mexican,39.721966,-86.153505,"311 Troy Ct, Indianapolis, IN 46227, USA",11:00 AM,11:00 PM,-0.607057
3,Tapatia Mexican Tacos,Mexican,39.772727,-86.040069,"7202 E Washington St, Indianapolis, IN 46219, USA",12:00 PM,11:00 PM,-1.067584
4,Traveling Taste Buds,Comfort Food,39.768534,-86.158142,"1 Monument Cir, Indianapolis, IN 46204, USA",12:00 AM,12:00 AM,-1.067584
5,Ice Cream On Wheels,Ice Cream,39.691830,-86.234110,"4215 W Thompson Rd, Indianapolis, IN 46221, USA",12:00 AM,12:00 AM,-1.067584
6,Tacopolis,Mexican,39.823769,-86.242204,"4857 W 38th St, Indianapolis, IN 46254, USA",02:00 PM,11:30 PM,-0.804426
7,TACO LOCO M&C,Mexican,39.911651,-86.184593,"1415 W 86th St, Indianapolis, IN 46260, USA",06:00 PM,12:00 AM,-1.067584
8,Scout's Treat Truck,Bakeries,39.908417,-86.082016,"8375 Allison Pointe Blvd, Indianapolis, IN 46250, USA",06:30 AM,08:00 PM,0.104964
9,Pico De Gallo,Mexican,39.853872,-86.231604,"4202 W 56th St, Indianapolis, IN 46254, USA",05:00 AM,01:00 PM,-0.664773


The plan will ocnsist of going to a different truck for breakfast, lunch and dinner.

In order to do so, I am creating seperate dataframes for each mean, filtering by the trucks that are open during the typical meal time

In [99]:
# Defining Mean Times
breakfast_time = pd.to_datetime("09:00 AM", format='%I:%M %p')
lunch_time = pd.to_datetime("01:00 PM", format='%I:%M %p')
dinner_time = pd.to_datetime("07:00 PM", format='%I:%M %p')

# Converting 'Start_Time' and 'End_Time' Columns to Datetime Format
rated_trucks_df['Start_Time'] = pd.to_datetime(rated_trucks_df['Start_Time'], format='%I:%M %p')
rated_trucks_df['End_Time'] = pd.to_datetime(rated_trucks_df['End_Time'], format='%I:%M %p')



# Creating Separate DataFrames for Breakfast, Lunch, and Dinner
breakfast_df = rated_trucks_df[(rated_trucks_df['Start_Time']
                                <= breakfast_time) & (rated_trucks_df['End_Time']
                                                      >= breakfast_time)]
lunch_df = rated_trucks_df[(rated_trucks_df['Start_Time']
                            <= lunch_time) & (rated_trucks_df['End_Time']
                                              >= lunch_time)]
dinner_df = rated_trucks_df[(rated_trucks_df['Start_Time']
                             <= dinner_time) & (rated_trucks_df['End_Time']
                                                >= dinner_time)]

# Sort the Filtered DataFrames by Normalized Rating
breakfast_df = breakfast_df.sort_values(by='Normalized_Ratings', ascending=False).reset_index(drop=True)
lunch_df = lunch_df.sort_values (by='Normalized_Ratings', ascending=False).reset_index(drop=True)
dinner_df = dinner_df.sort_values(by='Normalized_Ratings', ascending=False).reset_index(drop=True)

# Formatting Columns to Desired Time Format
breakfast_df['Start_Time'] = breakfast_df['Start_Time'].dt.strftime('%I:%M %p')
lunch_df['Start_Time'] = lunch_df['Start_Time'].dt.strftime('%I:%M %p')
dinner_df['Start_Time'] = dinner_df['Start_Time'].dt.strftime('%I:%M %p')

breakfast_df['End_Time'] = breakfast_df['End_Time'].dt.strftime('%I:%M %p')
lunch_df['End_Time'] = lunch_df['End_Time'].dt.strftime('%I:%M %p')
dinner_df['End_Time'] = dinner_df['End_Time'].dt.strftime('%I:%M %p')



# Select the top-rated food trucks for each meal
best_breakfast_trucks = breakfast_df['Food_Truck'].head(5)
best_lunch_trucks = lunch_df['Food_Truck'].head(5)
best_dinner_trucks = dinner_df['Food_Truck'].head(5)

# Display the selected food trucks
print("Best Breakfast Food Trucks:")
print(best_breakfast_trucks)

print("\nBest Lunch Food Trucks:")
print(best_lunch_trucks)

print("\nBest Dinner Food Trucks:")
print(best_dinner_trucks)

Best Breakfast Food Trucks:
0         Black Leaf Vegan
1      Scout's Treat Truck
2            Pico De Gallo
3    The Legendary Kitchen
Name: Food_Truck, dtype: object

Best Lunch Food Trucks:
0        Gaucho's Fire Express
1             Black Leaf Vegan
2          Scout's Treat Truck
3    Poccadio Grill Food Truck
4                   La Familia
Name: Food_Truck, dtype: object

Best Dinner Food Trucks:
0        Gaucho's Fire Express
1             BB&B Bread Truck
2                      Pi Indy
3          Scout's Treat Truck
4    Poccadio Grill Food Truck
Name: Food_Truck, dtype: object


In [100]:
# Create a list of selected food trucks
selected_food_trucks = [breakfast_df, lunch_df, dinner_df]

# Ensure that the selected food trucks have different cuisines
selected_cuisines = set()

final_selection = []
for food_trucks in selected_food_trucks:
    for _, row in food_trucks.iterrows():
        if row['Cuisine'] not in selected_cuisines:
            final_selection.append(row)
            selected_cuisines.add(row['Cuisine'])
            if len(final_selection) == 6:
                 break
        if len(final_selection) == 6:
         break

# Convert the final selection to a DataFrame
final_selection_df = pd.DataFrame(final_selection).reset_index(drop = True)

In [101]:
# Viewing the Resulting DataFrame
display_final_selection = final_selection_df
html_5 = scroll_table(display_final_selection, 'final_selection_df_html', "Selected Truck Data")
display(HTML(html_5))

,Food_Truck,Cuisine,Latitude,Longitude,Address,Start_Time,End_Time,Normalized_Ratings
0,Black Leaf Vegan,Comfort Food,39.779130,-86.164590,"335 W 9th St, Indianapolis, IN 46202, USA",08:00 AM,06:00 PM,2.514354
1,Scout's Treat Truck,Bakeries,39.908417,-86.082016,"8375 Allison Pointe Blvd, Indianapolis, IN 46250, USA",06:30 AM,08:00 PM,0.104964
2,Pico De Gallo,Mexican,39.853872,-86.231604,"4202 W 56th St, Indianapolis, IN 46254, USA",05:00 AM,01:00 PM,-0.664773
3,Gaucho's Fire Express,Sandwiches,39.784037,-86.163322,"1280 N Senate Ave, Indianapolis, IN 46202, USA",10:30 AM,09:00 PM,3.539175
4,Pi Indy,Pizza,39.789676,-86.306666,"8222 Indy Ln, Indianapolis, IN 46214, USA",05:00 PM,08:00 PM,0.988038


Unfortunately, the loop only returned 5 Food Trucks, due to the shortage of cuisines available, during required times. I will choose the final truck manually.
* Breakfast trucks:
  * Scout's Treat Truck
  * Pico De Gallo
* Lunch Trucks:
  * Black Leaf Vegan
  * Gaucho's Fire Express
* Dinner Truck:
  * Pi Indy
  * BB&B Bread Truck

As we needed another truck, I chose BB&B Bread Truck, as it is the best dinner truck not already in the selection

In [102]:
# Selecting the Trucks for Day 1 and 2
Day_1_trucks = rated_trucks_df.query("Food_Truck == 'Pico De Gallo' | Food_Truck == 'Black Leaf Vegan' | Food_Truck == 'BB&B Bread Truck'").reset_index(drop=True)
Day_2_trucks = rated_trucks_df.query("Food_Truck == \"Scout's Treat Truck\" | Food_Truck == \"Gaucho's Fire Express\" | Food_Truck == 'Pi Indy'").reset_index(drop=True)

# Converting Start and End Times to Desired Time Format
Day_1_trucks['Start_Time'] = Day_1_trucks['Start_Time'].dt.strftime('%I:%M %p')
Day_1_trucks['End_Time'] = Day_1_trucks['End_Time'].dt.strftime('%I:%M %p')
Day_2_trucks['Start_Time'] = Day_2_trucks['Start_Time'].dt.strftime('%I:%M %p')
Day_2_trucks['End_Time'] = Day_2_trucks['End_Time'].dt.strftime('%I:%M %p')

In [103]:
# Viewing the Resulting DataFrame
display_day_1_trucks_df = Day_1_trucks
html_6 = scroll_table(display_day_1_trucks_df, 'day_1_trucks_df_html', "Truck Data for the 1st Day of the Plan")
display(HTML(html_6))

,Food_Truck,Cuisine,Latitude,Longitude,Address,Start_Time,End_Time,Normalized_Ratings
0,Pico De Gallo,Mexican,39.853872,-86.231604,"4202 W 56th St, Indianapolis, IN 46254, USA",05:00 AM,01:00 PM,-0.664773
1,BB&B Bread Truck,Bakeries,39.779108,-86.163845,"800 Lafayette St, Indianapolis, IN 46202, USA",05:00 PM,10:00 PM,1.229964
2,Black Leaf Vegan,Comfort Food,39.779130,-86.164590,"335 W 9th St, Indianapolis, IN 46202, USA",08:00 AM,06:00 PM,2.514354


As the lunch and dinner rows are switched, I will interchange them, for future needs of mapping and getting travel data

In [104]:
# Switching 2nd and 3rd Rows
Day_1_trucks.iloc[1], Day_1_trucks.iloc[2] = Day_1_trucks.iloc[2].copy(), Day_1_trucks.iloc[1].copy()

In [105]:
# Viewing the Resulting DataFrame
display_day_1_corrected_trucks_df = Day_1_trucks
html_7 = scroll_table(display_day_1_corrected_trucks_df, 'day_1_trucks_corrected_df_html', "Corrected Truck Data for the 1st Day of the Plan")
display(HTML(html_7))

,Food_Truck,Cuisine,Latitude,Longitude,Address,Start_Time,End_Time,Normalized_Ratings
0,Pico De Gallo,Mexican,39.853872,-86.231604,"4202 W 56th St, Indianapolis, IN 46254, USA",05:00 AM,01:00 PM,-0.664773
1,Black Leaf Vegan,Comfort Food,39.779130,-86.164590,"335 W 9th St, Indianapolis, IN 46202, USA",08:00 AM,06:00 PM,2.514354
2,BB&B Bread Truck,Bakeries,39.779108,-86.163845,"800 Lafayette St, Indianapolis, IN 46202, USA",05:00 PM,10:00 PM,1.229964


In [106]:
# Viewing the Resulting DataFrame
display_day_2_trucks_df = Day_2_trucks
html_8 = scroll_table(display_day_2_trucks_df, 'day_2_trucks_df_html', "Truck Data for the 2nd Day of the Plan")
display(HTML(html_8))

,Food_Truck,Cuisine,Latitude,Longitude,Address,Start_Time,End_Time,Normalized_Ratings
0,Scout's Treat Truck,Bakeries,39.908417,-86.082016,"8375 Allison Pointe Blvd, Indianapolis, IN 46250, USA",06:30 AM,08:00 PM,0.104964
1,Gaucho's Fire Express,Sandwiches,39.784037,-86.163322,"1280 N Senate Ave, Indianapolis, IN 46202, USA",10:30 AM,09:00 PM,3.539175
2,Pi Indy,Pizza,39.789676,-86.306666,"8222 Indy Ln, Indianapolis, IN 46214, USA",05:00 PM,08:00 PM,0.988038


To find travel information, I have set the starting address as the ofiice of TechPoint, Indianapolis. I mainly wanted to bicycling, transit or rideshare (Uber) as the modes of transport, as if the Xterns execute this plan, those will be the best modes of travel.

In [ ]:
# Scraping Travel Data

# Defining the Starting Address
starting_address = "1210 Waterway Blvd Suite 5000, Indianapolis, IN 46202" # TechPoint Indianapolis Address

# Creating a Function to get Travel Information
def get_travel_info(destination_address):
    try:
        # Use the Google Maps Directions API to get travel information
        directions = gmaps_client.directions(starting_address, destination_address, mode="driving")

        if directions:
            route = directions[0]['legs'][0]
            travel_distance = route['distance']['value']  # Get distance in meters

          # Setting Transport type Parameters
            if travel_distance <= 2000:
              mode = "bicycling"
            elif travel_distance >= 10000:
              mode = "transit"
            else:
              mode = "rideshare"

            travel_time = route['duration']['text']
            transportation_type = mode

            return travel_time, travel_distance, transportation_type
        else:
            return None, None, None
    except Exception as e:
        print(f"Error: {e}")
        return None, None, None

# Iterating Through the DataFrame and Calculating Travel Information
for index, row in Day_1_trucks.iterrows():
    destination_address = row['Address']
    travel_time, travel_distance, transportation_type = get_travel_info(destination_address)

    Day_1_trucks.at[index, 'Travel_Time'] = travel_time
    Day_1_trucks.at[index, 'Travel_Distance'] = travel_distance
    Day_1_trucks.at[index, 'Transportation_Type'] = transportation_type

# Updating DataFrames
Day_1_plan = Day_1_trucks
Day_1_clean_plan = Day_1_trucks[["Food_Truck", "Start_Time", "End_Time", "Cuisine", "Address", "Travel_Time", "Travel_Distance", "Transportation_Type"]]
Day_1_clean_plan["Time"] = [breakfast_time, lunch_time, dinner_time]
Day_1_clean_plan["Time"] = Day_1_clean_plan["Time"].dt.strftime('%I:%M %p')

# Converting Meters to Miles

# Defining a Function for Conversion
def meters_to_miles(meters):
    miles = meters * 0.000621371  # 1 meter is approximately 0.000621371 miles
    return miles

# Applying the Function to the Clean DataFrame
Day_1_clean_plan['Travel_Distance'] = Day_1_clean_plan['Travel_Distance'].apply(meters_to_miles)

def format_distance(distance):
    formatted_distance = f"{distance:.2f} Miles"
    return formatted_distance

# Apply the function to the 'Distance_Miles' column
Day_1_clean_plan['Travel_Distance'] = Day_1_clean_plan['Travel_Distance'].apply(format_distance)


In [122]:
# Shifting Time column to the Front of the DataFrame

time_column = Day_1_clean_plan["Time"]

# Removing the last column from the DataFrame
Day_1_clean_plan = Day_1_clean_plan.drop(columns = "Time")

# Insert the last column at the beginning of the DataFrame
Day_1_clean_plan.insert(0, 'Time', time_column)

# Saving Day 2 Plan to Excel
Day_1_clean_plan.to_excel('Day_1_Plan.xlsx', index = False)

In [123]:
# Viewing the Resulting DataFrame
display_day_1_plan_df = Day_1_clean_plan
html_9 = scroll_table(display_day_1_plan_df, 'day_1_trucks_plan_html', "Information about the 1st Day of the Plan")
display(HTML(html_9))

,Time,Food_Truck,Start_Time,End_Time,Cuisine,Address,Travel_Time,Travel_Distance,Transportation_Type
0,09:00 AM,Pico De Gallo,05:00 AM,01:00 PM,Mexican,"4202 W 56th St, Indianapolis, IN 46254, USA",14 mins,6.76 Miles,transit
1,01:00 PM,Black Leaf Vegan,08:00 AM,06:00 PM,Comfort Food,"335 W 9th St, Indianapolis, IN 46202, USA",8 mins,1.60 Miles,rideshare
2,07:00 PM,BB&B Bread Truck,05:00 PM,10:00 PM,Bakeries,"800 Lafayette St, Indianapolis, IN 46202, USA",8 mins,1.63 Miles,rideshare


In [ ]:
# Iterate through the DataFrame and calculate travel information
for index, row in Day_2_trucks.iterrows():
    destination_address = row['Address']
    travel_time, travel_distance, transportation_type = get_travel_info(destination_address)

    Day_2_trucks.at[index, 'Travel_Time'] = travel_time
    Day_2_trucks.at[index, 'Travel_Distance'] = travel_distance
    Day_2_trucks.at[index, 'Transportation_Type'] = transportation_type

# Print the updated DataFrame
Day_2_plan = Day_2_trucks
Day_2_clean_plan = Day_2_trucks[["Food_Truck", "Start_Time", "End_Time", "Cuisine", "Address", "Travel_Time", "Travel_Distance", "Transportation_Type"]]
Day_2_clean_plan["Time"] = [breakfast_time, lunch_time, dinner_time]
Day_2_clean_plan["Time"] = Day_2_clean_plan["Time"].dt.strftime('%I:%M %p')

# Converting Meters to Miles

# Defining a Function for Conversion
def meters_to_miles(meters):
    miles = meters * 0.000621371  # 1 meter is approximately 0.000621371 miles
    return miles

# Applying the Function to the Clean DataFrame
Day_2_clean_plan['Travel_Distance'] = Day_2_clean_plan['Travel_Distance'].apply(meters_to_miles)

def format_distance(distance):
    formatted_distance = f"{distance:.2f} Miles"
    return formatted_distance

# Apply the function to the 'Distance_Miles' column
Day_2_clean_plan['Travel_Distance'] = Day_2_clean_plan['Travel_Distance'].apply(format_distance)

# Shifting the Time Column to the Front of the DataFrame
time_column = Day_2_clean_plan["Time"]

# Remove the last column from the DataFrame
Day_2_clean_plan = Day_2_clean_plan.drop(columns = "Time")

# Insert the last column at the beginning of the DataFrame
Day_2_clean_plan.insert(0, 'Time', time_column)

# Saving Day 2 Plan to Excel
Day_2_clean_plan.to_excel('Day_2_Plan.xlsx', index = False)

In [121]:
# Viewing the Resulting DataFrame
display_day_2_plan_df = Day_2_clean_plan
html_10 = scroll_table(display_day_2_plan_df, 'day_2_trucks_plan_html', "Information about the 2nd Day of the Plan")
display(HTML(html_10))

,Time,Food_Truck,Start_Time,End_Time,Cuisine,Address,Travel_Time,Travel_Distance,Transportation_Type
0,09:00 AM,Scout's Treat Truck,06:30 AM,08:00 PM,Bakeries,"8375 Allison Pointe Blvd, Indianapolis, IN 46250, USA",27 mins,18.62 Miles,transit
1,01:00 PM,Gaucho's Fire Express,10:30 AM,09:00 PM,Sandwiches,"1280 N Senate Ave, Indianapolis, IN 46202, USA",7 mins,1.65 Miles,rideshare
2,07:00 PM,Pi Indy,05:00 PM,08:00 PM,Pizza,"8222 Indy Ln, Indianapolis, IN 46214, USA",18 mins,7.55 Miles,transit


In [112]:
# Getting Coordinates of Starting Address

geocode_result = gmaps_client.geocode(starting_address)
if geocode_result:
    start_location = geocode_result[0]['geometry']['location']
    start_latitude = start_location['lat']
    start_longitude = start_location['lng']

starting_coordinates = (start_latitude, start_longitude)

In [113]:
# Importing Required Visualization Packages

import folium
from geopy.distance import geodesic

In [114]:
# Day 1 Plan Visualization

# Creating a Map Centered at the Starting Point
map_1 = folium.Map(location=starting_coordinates, zoom_start=13)

# Adding a Marker for the Starting Point
folium.Marker(
    location=starting_coordinates,
    popup="Starting Point",
    icon=folium.DivIcon(html='<div style="font-size: 18px;">Start</div>')
).add_to(map_1)

# Adding Markers for Food Truck Locations
for index, row in Day_1_plan.iterrows():
    food_truck_coordinates = (row['Latitude'], row['Longitude'])
    food_truck_name = row['Food_Truck']
    travel_distance = row['Travel_Distance']

    # Creating a Polyline Between the Starting Point and Food Truck Locations
    folium.PolyLine(
        locations=[starting_coordinates, food_truck_coordinates],
        color='red',
        weight=2.5,
        opacity=1.0,
    ).add_to(map_1)

    # Adding Markers for the Food Truck Locations
    destination_number = str(index + 1)
    folium.Marker(
        location=food_truck_coordinates,
        popup=food_truck_name,
        icon=folium.DivIcon(html=f'<div style="font-size: 16px;">{destination_number}</div>')
    ).add_to(map_1)

# Saving Map to HTML
map_1.save("Plan_1_map.html")

In [118]:
# Day 2 Plan Visualization

# Creating a Map Centered at the Starting Point
map_2 = folium.Map(location=starting_coordinates, zoom_start=13)

# Adding a Marker for the Starting Point
folium.Marker(
    location=starting_coordinates,
    popup="Starting Point",
    icon=folium.DivIcon(html='<div style="font-size: 18px;">Start</div>')
).add_to(map_2)

# Adding Markers for Food Truck Locations
for index, row in Day_2_plan.iterrows():
    food_truck_coordinates = (row['Latitude'], row['Longitude'])
    food_truck_name = row['Food_Truck']
    travel_distance = row['Travel_Distance']

    # Creating a Polyline Between the Starting Point and Food Truck Locations
    folium.PolyLine(
        locations=[starting_coordinates, food_truck_coordinates],
        color='red',
        weight=2.5,
        opacity=1.0,
    ).add_to(map_2)

    # Adding Markers for the Food Truck Locations
    destination_number = str(index + 1)
    folium.Marker(
        location=food_truck_coordinates,
        popup=food_truck_name,
        icon=folium.DivIcon(html=f'<div style="font-size: 16px;">{destination_number}</div>')
    ).add_to(map_2)

# Saving Map to HTML
map_2.save("Plan_2_map.html")

In [124]:
# Downloading Plan 1
files.download("Day_1_Plan.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [125]:
# Downloading Plan 2
files.download("Day_2_Plan.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [116]:
# Downloading Map 1
files.download("Plan_1_map.html")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [119]:
# Downloading Map 2
files.download("Plan_2_map.html")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>